In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image

from scipy import special
import pandas as pd
import numpy as np
import cv2
import os

In [ ]:
def eye_hw(lmk):
    def Euclidian_distant(xy1, xy2):
        return ((xy1[0]-xy2[0])**2+(xy1[1]-xy2[1])**2)**0.5
    
    left_eye = lmk[33:52]
    right_eye = lmk[87:]
    centre_nose = lmk[86]

    left_brow = left_eye[-9:]
    right_brow = right_eye[-9:]    
    left_eye = left_eye[:-9]
    right_eye = right_eye[:-9]

    left_eye_max_h = Euclidian_distant(left_eye[0], left_eye[7])
    left_eye_h = 1/3*(
                 Euclidian_distant(left_eye[3], left_eye[8])+
                 Euclidian_distant(left_eye[0], left_eye[7])+
                 Euclidian_distant(left_eye[4], left_eye[9]))
    left_eye_w = Euclidian_distant(left_eye[2], left_eye[6])

    right_eye_max_h = Euclidian_distant(right_eye[0], right_eye[7])    
    right_eye_h = 1/3*(
                 Euclidian_distant(right_eye[3], right_eye[8])+
                 Euclidian_distant(right_eye[0], right_eye[7])+
                 Euclidian_distant(right_eye[4], right_eye[9]))
    right_eye_w = Euclidian_distant(right_eye[2], right_eye[6])

    hw_left = [left_eye_max_h, left_eye_h, left_eye_w]                                  
    hw_right = [right_eye_max_h, right_eye_h, right_eye_w]  
    return hw_left, hw_right

In [ ]:
def const_faces(df):
    app = FaceAnalysis(allowed_modules=['detection', 'landmark_2d_106'])
    app.prepare(ctx_id=0, det_size=(192, 192))

    for index, row in df.iterrows():
        print(index)
        if index < 550000:
            continue
        file = row['file']
        confidence = row['confidence']
        
        if confidence >= 0:
            img = cv2.imread(file)
            faces = app.get(img)
            if len(faces)==1:
                for face in faces:
                    lmk = face.landmark_2d_106
                    lmk = np.round(lmk).astype(np.int)
                    hw_left, hw_right = eye_hw(lmk)

                    const_l = round(hw_left[1]/hw_left[2], 2)
                    const_r = round(hw_right[1]/hw_right[2], 2)

            else:
                const_l = -1
                const_r = -1
        else:
            const_l = -1
            const_r = -1
        df.loc[index, 'const_left'] = const_l
        df.loc[index, 'const_right'] = const_r
        #if index == 650000:
        #    df.to_csv('/home/yandex/igor/data_eyes/video_frames_const.csv', sep=';',index=False)
        #    break
            
        #const_l_list.append(const_l)
        #const_r_list.append(const_r)

    #df['const_left'] = const_l_list
    #df['const_right'] = const_r_list
    return df

In [ ]:
#fairface_df = pd.read_csv('/home/yandex/igor/data_eyes/video_frames.csv', sep=';')
#fairface_df['const_left'] = [-1 for i in range(len(fairface_df))]
#fairface_df['const_right'] = [-1 for i in range(len(fairface_df))]

fairface_df = pd.read_csv('/home/yandex/igor/data_eyes/video_frames_const.csv', sep=';')
const_df = const_faces(fairface_df)
const_df.to_csv('/home/yandex/igor/data_eyes/video_frames_const.csv', sep=';',index=False)

In [ ]:
#def append_ext(fn):
#    img_path = '/home/yandex/igor/data_eyes/altyn_crops'
#    return img_path + '/' + fn.split('/')[-1]

#fairface_df['file']=fairface_df['file'].apply(append_ext)
#fairface_df

In [ ]:
#df = const_faces(fairface_df)
#df

In [ ]:
def genome_faces(df):
    """
    asia_index = 0
    euro_index = 1
    afro_index = 2
    """
    index_to_race = ['asian', 'european', 'african']
    export_dir = '/home/yandex/igor/sensor_models/genome/0003'
    genome_model = tf.saved_model.load(export_dir)

    genome_list = []
    for index, row in df.iterrows():
        file = row['file']
        confidence = row['confidence']
            
        if confidence >= 0:
            images_bytes = [open(file, "rb").read()]
            output = genome_model.signatures['serving_default'](tf.convert_to_tensor(images_bytes))["confidence"]
            ind = (output.numpy()[0]).argmax(axis=0)
            race = index_to_race[ind]
            genome_list.append(race)
        else:
            print('confidence = ', confidence)
            genome_list.append('alien')
    df['genome'] = genome_list
    return df

#df = genome_faces(fairface_df)
#df

In [ ]:
"""export_dir = '/home/yandex/igor/sensor_models/genome/0003'
genome_model = tf.saved_model.load(export_dir)

genome_list = []
for index, row in fairface_df[:1].iterrows():
    file = row['file']
    confidence = row['confidence']

    images_bytes = [open(file, "rb").read()]
    output = genome_model.signatures['serving_default'](tf.convert_to_tensor(images_bytes))["confidence"]
    print((output.numpy()[0]).argmax(axis=0))
    print(file)"""


In [ ]:
#df['genome'].value_counts()

In [ ]:
#df.to_csv('/home/yandex/igor/data_eyes/Halyk_const_genome.csv', sep=';',index=False)

In [ ]:
def const_faces_add(df):
    app = FaceAnalysis(allowed_modules=['detection', 'landmark_2d_106'])
    app.prepare(ctx_id=0, det_size=(192, 192))
    cnt = 0
    const_l_list = []
    const_r_list = []
    for index, row in df.iterrows():
        file = row['file']
        confidence = row['confidence']
        const_l = row['const_left']
        const_r = row['const_right']
        
        if confidence == 0:
            img = cv2.imread(file)
            faces = app.get(img)
            if len(faces)==1:
                for face in faces:
                    lmk = face.landmark_2d_106
                    lmk = np.round(lmk).astype(np.int)
                    hw_left, hw_right = eye_hw(lmk)

                    const_l = round(hw_left[1]/hw_left[2], 2)
                    const_r = round(hw_right[1]/hw_right[2], 2)
                print('change lmk')
                cnt += 1
            else:
                const_l = -1
                const_r = -1
        const_l_list.append(const_l)
        const_r_list.append(const_r)

    df['const_left'] = const_l_list
    df['const_right'] = const_r_list
    print('cnt = ', cnt)
    return df
#df_fixed = const_faces_add(df)
#df_fixed

In [ ]:
def podium_to_df(df):
    export_dir = '/home/yandex/igor/sensordev/sensordev_runner/0003rules'
    detector = tf.saved_model.load(export_dir)
    for index, row in df.iterrows():
        filename = row['file']

        with open(filename, "rb") as f:
            image_bytes = f.read()
            output = detector.signatures["serving_default"](tf.convert_to_tensor([image_bytes], tf.string))


In [ ]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

df_test = pd.read_csv('/home/yandex/igor/data_eyes/tags_57_71_const_genome.csv', sep=';')

export_dir = '/home/yandex/igor/sensordev/sensordev_runner/0003rules'
detector = tf.saved_model.load(export_dir)

genome_list = []
for index, row in df_test.iterrows():
    file = row['filename']
        
    images_bytes = [open(file, "rb").read()]
    output = detector.signatures["serving_default"](tf.convert_to_tensor([images_bytes], tf.string))
    print(output['blink'])

In [ ]:
def genome_faces(df):

    export_dir = '/home/yandex/igor/sensordev/sensordev_runner/0003rules'
    detector = tf.saved_model.load(export_dir)

    genome_list = []
    for index, row in df.iterrows():
        file = row['filename']
            
        images_bytes = [open(file, "rb").read()]
        output = detector.signatures["serving_default"](tf.convert_to_tensor([images_bytes], tf.string))
        print(output['blink'])

        #genome_list.append(race)

    #df['genome'] = genome_list
    return df
df_test = pd.read_csv('/home/yandex/igor/data_eyes/tags_57_71_const_genome.csv', sep=';')

df = genome_faces(df_test)
df